In [127]:
import os.path, csv
from pprint import pprint
from math import copysign, sqrt
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual

simulation_names = [name for name in os.listdir("gatling-results") if os.path.isfile(os.path.join("gatling-results", name, "simulation.log"))]

In [128]:
def init_bucket():
    return dict(
                requests_per_second = 0,
                cumulative_count = 0,
                active_user_count = 0,
                count = 0,
                mean_rtime = 0,
                M2 = 0.,
                min_rtime = float("+inf"),
                max_rtime = float("-inf"),
                median_avg = 0.,
                median_rtime = 0.,
            )

def update_bucket(bucket, sample, total_n_samples):
    bucket['cumulative_count'] = total_n_samples 
    #Update extremes
    bucket['min_rtime'] = min(bucket['min_rtime'], sample)
    bucket['max_rtime'] = max(bucket['max_rtime'], sample)

    #Update mean/stdev (Welford's Algorithm)
    count, mean, M2 = bucket['count'], bucket['mean_rtime'], bucket['M2']
    count += 1
    delta = sample - mean
    mean += delta / count
    delta2 = sample - mean
    M2 += delta * delta2
    bucket['M2'] = M2
    bucket['count'] = count
    bucket['mean_rtime'] = mean

    #Update median (Jeff McClintock estimate)
    bucket['median_avg'] += (sample - bucket['median_avg']) * 0.1
    bucket['median_rtime'] += copysign( bucket['median_avg'] * 0.01, sample - bucket['median_rtime'] );

def finalize_bucket(bucket, length_seconds):
    if length_seconds:
        bucket["requests_per_second"] = bucket["count"] *1. / length_seconds
    count, mean, M2 = bucket['count'], bucket['mean_rtime'], bucket['M2']
    del bucket['M2']
    del bucket['median_avg']
    if count > 1:
        (mean, variance, sampleVariance) = (mean, M2/count, M2/(count - 1)) 
        bucket['stdev_rtime'] = sqrt(M2/count)
    else:
        bucket['stdev_rtime'] = float("nan")

def calculate_stats(name):
    print ("Processing log file for test run {}".format(name))
    user_add_timestamps = set()
    time_buckets = {}
    total_stats = dict(
                total = init_bucket(),
                fail = init_bucket(),
                success = init_bucket()
            )
    total_counts = dict(
        total = 0,
        fail = 0,
        success = 0,
    )
    
    def process_user(scenario, userid, action, start_ts, end_ts):
        if action != "START":
            raise RuntimeError("Don't know what to do with USER line with action {}".format(action))
        if start_ts != end_ts:
            raise RuntimeError("Don't know how to handle USER START messages with start/end timestamps {} / {}.".format(start_ts, end_ts))
        user_add_timestamps.add(int(start_ts)//1000)
    
    def process_request(user, _1, request, start_ts, end_ts, result, _2):
        start_ts = float(start_ts)
        end_ts = float(end_ts)
        dur = end_ts - start_ts
        bucket_index = int(end_ts//1000)
        success = result == "OK"
        total_counts['total'] += 1
        if success:
            total_counts['success'] += 1
        else:
            total_counts['fail'] += 1
            
        if bucket_index not in time_buckets:
            time_buckets[bucket_index] = dict(
                total = init_bucket(),
                fail = init_bucket(),
                success = init_bucket()
            )
            
        update_bucket(total_stats['total'], dur, total_counts['total'])
        update_bucket(time_buckets[bucket_index]['total'], dur, total_counts['total'])
        if success:
            update_bucket(time_buckets[bucket_index]['success'], dur, total_counts['success'])
            update_bucket(total_stats['success'], dur, total_counts['success'])
        else:
            update_bucket(time_buckets[bucket_index]['fail'], dur, total_counts['fail'])
            update_bucket(total_stats['fail'], dur, total_counts['fail'])
        
    
    with open(os.path.join("gatling-results", name, "simulation.log")) as inf:
        for line in inf:
            line = line.rstrip("\n")
            type, *args = line.split("\t")
            if type == "RUN":
                continue
            elif type == "USER":
                process_user(*args)
            elif type == "REQUEST":
                if len(args) != 7:
                    print("Don't know how to handle REQUEST log {}".format(args))
                else:
                    process_request(*args)
            else:
                print(type, args)
        
    #Fix data for times when gatling logs provided nothing at at all:
    for ts in range(min(time_buckets), max(time_buckets)):
        if ts not in time_buckets:
            time_buckets[ts] = dict(total = init_bucket(), fail = init_bucket(), success = init_bucket())
            for prev_ts in range(min(time_buckets), ts):
                for key in ['total', 'fail', 'success']:
                    time_buckets[ts][key]['cumulative_count'] = time_buckets[prev_ts][key]['cumulative_count']
        
    for add_ts in sorted(user_add_timestamps):
        total_stats['total']['active_user_count'] += 1
        total_stats['fail']['active_user_count'] += 1
        total_stats['success']['active_user_count'] += 1
        for bucket_ts in sorted(time_buckets):
            if bucket_ts >= add_ts:
                time_buckets[bucket_ts]['total']['active_user_count'] += 1
                time_buckets[bucket_ts]['fail']['active_user_count'] += 1
                time_buckets[bucket_ts]['success']['active_user_count'] += 1
        
    test_duration = max(time_buckets) - min(time_buckets)
    finalize_bucket(total_stats['total'], test_duration)
    finalize_bucket(total_stats['fail'], test_duration)
    finalize_bucket(total_stats['success'], test_duration)
    for bucket_ts in sorted(time_buckets):
        finalize_bucket(time_buckets[bucket_ts]['total'], 1)
        finalize_bucket(time_buckets[bucket_ts]['fail'], 1)
        finalize_bucket(time_buckets[bucket_ts]['success'],1 )
        
    return total_stats, time_buckets
        
    
simulations = {}
for name in simulation_names:
    stats_total, stats_by_time = calculate_stats(name)
    simulations[name] = dict(
        stats_total = stats_total,
        stats_by_time = stats_by_time,
        start_idx = sorted(stats_by_time)[0]
    )

Processing log file for test run primefactorkata-20191108095935189
Processing log file for test run primefactorkata-20191108095117147


In [129]:
@interact
def plot_stats_over_time(name = sorted(simulation_names), field = simulations[simulation_names[0]]['stats_total']['total'].keys()):
    sim = simulations[name]
    stats = sim['stats_by_time']
    n_xs = 1+max(stats.keys()) - min(stats.keys())
    xs = np.arange(n_xs)
    fail_y = np.zeros(n_xs)
    success_y = np.zeros(n_xs) 
    total_y = np.zeros(n_xs)
     
    max_val = float("-inf")
    for ts in sorted(stats):
        idx = ts - min(stats.keys())
        fail_y[idx] = stats[ts]['fail'][field]
        total_y[idx] = stats[ts]['total'][field]
        success_y[idx] = stats[ts]['success'][field]
        max_val = max(max_val, fail_y[idx], total_y[idx], success_y[idx])
        
    plt.rcParams["figure.figsize"] = (20,10)
    plt.xlim(0, n_xs)
    plt.ylim(0, 1.2 * max_val)
    plt.plot(xs, fail_y, label="Failed", color="red")
    plt.plot(xs, success_y, label="OK", color="green")
    plt.plot(xs, total_y, label="Total", color="blue")
    
    plt.legend()
    plt.show()

interactive(children=(Dropdown(description='name', options=('primefactorkata-20191108095117147', 'primefactork…

In [130]:
@interact
def plot_comparison(field = simulations[simulation_names[0]]['stats_total']['total'].keys()):
    names = []
    values = []
    for name in sorted(simulation_names):
        v = simulations[name]['stats_total']['total'][field]
        names.append("{}\n{:.2f}".format(name,v))
        values.append(v)
        

    x = np.arange(len(names))
    plt.bar(x, values)
    plt.xticks(x, names)
    plt.show()

interactive(children=(Dropdown(description='field', options=('requests_per_second', 'cumulative_count', 'activ…